In [1]:
from huggingface_hub import notebook_login
import pandas as pd

In [2]:
#notebook_login()

In [3]:
#doesn't seem to run  for some reason

from datasets import load_dataset
reddit = load_dataset('reddit')

Found cached dataset reddit (C:/Users/AT030915/.cache/huggingface/datasets/reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e)


  0%|          | 0/1 [00:00<?, ?it/s]

# Output saved
Selection deleted
#doesn't seem to run  for some reason

from datasets import load_dataset
reddit = load_dataset('reddit')

Downloading and preparing dataset reddit/default to C:/Users/AT030915/.cache/huggingface/datasets/reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e...

Downloading data:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3848330 [00:00<?, ? examples/s]

Dataset reddit downloaded and prepared to C:/Users/AT030915/.cache/huggingface/datasets/reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e. Subsequent calls will reuse this data.

  0%|          | 0/1 [00:00<?, ?it/s]

# Subset the data
Want a data set ~ 10k in size 
Will choose the first 10k league of legends subreddit entries


In [4]:
#just loop thorugh taking only the league of legends subreddit data
N = 10000
S_N = N/10
I = 0
J = 0
subreddit = 'leagueoflegends'
data_store = []

while(J < N):
    data = reddit['train'][I]
    data_bool = data['subreddit']
    if data_bool == subreddit:
        #print(J)
        J += 1
        data_store.append(data)

        if (J % S_N) ==0:
            print(J) 

    I += 1

        
    
    

1000
2000



KeyboardInterrupt



In [ ]:
df = pd.DataFrame(data_store)
df.to_csv(f'data/reddit_{subreddit}_N_{N}.csv')

# Now have a data set
Only need the body and summary columns

In [1]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [2]:
name = 'data/reddit_leagueoflegends_N_10000.csv'
df = pd.read_csv(name)

In [23]:
df_small = df[0:10]
name_small = 'data/reddit_leagueoflegends_N_10.csv'
df_small.to_csv(name_small)

In [24]:
# convert into data_loader object


reddit_lol1 = load_dataset("csv", data_files=name_small)
#need to test train spli tas well
reddit_lol = reddit_lol1['train'].train_test_split(test_size=0.3)
reddit_lol

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/AT030915/.cache/huggingface/datasets/csv/default-1ae881b243be8771/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
        num_rows: 7
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
        num_rows: 3
    })
})

In [25]:
checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [26]:
prefix = "summarize: " #choose prefix so t5 summarises

#define function to 
def preprocess_function(examples):
    main = 'body' #main field name
    summary = 'summary' #summary field name 
    
    inputs = [prefix + doc for doc in examples[main]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples[summary], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [27]:
tokenized_reddit_lol = reddit_lol.map(preprocess_function, batched=True)

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [28]:
# now use a data collator - which pads the data to the size of the largest entry

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [29]:
# now choose an evaluator for the model
rouge = evaluate.load("rouge")

In [30]:
# now we need a function that passes data to the evaulator 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [31]:
# now start the transfer learning
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [32]:
tokenized_reddit_lol

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3
    })
})

In [34]:
# Training Hyperparameters now 

training_args = Seq2SeqTrainingArguments(
    output_dir="reddit_lol_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    #save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False , #fp16:  set to false for CPU,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_reddit_lol["train"],
    eval_dataset=tokenized_reddit_lol["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [35]:
# train 

trainer.train()

C:\Users\AT030915\HuggingFace_Text_Summarisation\venv\HF_venv\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.832088,0.316300,0.237200,0.316300,0.316300,19.000000


TrainOutput(global_step=7, training_loss=0.9907541956220355, metrics={'train_runtime': 18.8336, 'train_samples_per_second': 0.372, 'train_steps_per_second': 0.372, 'total_flos': 680145420288.0, 'train_loss': 0.9907541956220355, 'epoch': 1.0})

In [40]:
file_path = 'reddit_lol_model/'
trainer.save_model(file_path)
    

# Inferencing

In [61]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

text_lol =""" They're probably starting Lvmao, former JDG, LNG and most recently BLG sub support. Lele played for RNG youth in the past and was on FPX last split before he got replaced by QiuQiu.
Not that many good support players available right now, so probably the best possible pick up for RNG considering budget etc.""" 

with open('league_art_2.txt','r') as f:
    text_lol_art = f.read()

text_lol_art

"A Reddit post pointed out the critical flaw of freezing top lane at the professional level.\n\n\n\nTSM's Kayys, a League of Legends Head Scout and Strategy Coach made multiple comments on Fudge's actions in the first minutes of C9's games against TL, creating a disagreement between 2 players at the professional level.\n\nAs everyone saw, C9 lost to TL 3 games to 1, and Fudge's lack of pressure in the beginning in the top lane may have contributed a considerable amount, even while playing a top-tier champion.\n\nThe main argument lies within the gold distribution. Fudge was able to farm risk-free for the most part, accumulating gold and experience faster than his enemy laner. However, TL gained the same amount, if not more, just spread over 4 or 5 members that balance with Fudge's peaceful farming.\n\n\n\nKnowledgeable League of Legends commentators mentioned how freezing a lane is more of an individual mindset. It can only work when the entire team synergies and creates a plan for eve

In [62]:
from transformers import pipeline

summarizer = pipeline("summarization", model=file_path,max_length = 300)
summarizer(text_lol_art)

[{'summary_text': "a reddit post pointed out the critical flaw of freezing top lane at the professional level . the main argument lies within the gold distribution . TL gained the same amount, if not more, just spread over 4 or 5 members that balance with Fudge's peaceful farming ."}]